# 2025-04-25: Cornell Pressure Sensor Test  
2025-04-25  
Project: misc  
Jonathan Pfeifer  

## Purpose
This is a retest of Nikolai's pressure sensor. I rebuilt the main module and will retest with some additional testing as well to get some characterization data out of it.

## Building the sensor
```{figure} ../images/jb/2025-04-25-decapped-picow.jpg
:name: 2025-04-25-figure-decapped-picow
The first step was to take off the shield on the wireless chip. We hypothesized that this shield might be the reason Nikolai's original failed. I took it upstairs to AVAST and and hit it with the reflow solder and took it off. I also took two small capacitors with it, but since I have no plan to use the wireless features I think this is okay.
```

Then I soldered all the wires and boards together. Unfortunately forgot to take pictures here. But I just followed the images Nikolai provided for assembly. 

```{figure} ../images/jb/2025-04-25-potting.jpg
:name: 2025-04-25-figure-potting
The next step was potting the electronics. I used a standard 4N resin and then pulled the strongest vacuum on it that I could (about -0.7bar). I cycled the vacuum on and off, and tapped the box back and forth to get as many air bubbles to come out as possible. After about 5 minutes of this I released the vacuum and allowed it to finish curing overnight
```

```{figure} ../images/jb/2025-04-25-attached-unit.jpg
:name: 2025-04-25-figure-attached-unit
My orientation is different than Nikolai's original. I flipped both the sensor and the magnet around. The sensor has such a short cable that I had to do it this way. 
```

## Code
Here is the app code I am using to collect all the data for these trials. It opens up a GUI you can use. (Note, apparently this app is super glitchy and isn't recording correctly. So just be aware of that. I have now had to go back and re-record data number of times which is frustrating)


In [1]:
#!/usr/bin/env python3

import serial
import csv
import datetime
import os
import tkinter as tk
from tkinter import ttk, messagebox
from serial.tools import list_ports
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from time import time

# --- USER SETTINGS ---
SAVE_DIR = "../data/raw/cornell-sensor/"
os.makedirs(SAVE_DIR, exist_ok=True)


class Plotter:
    def __init__(self, ax):
        self.ax = ax
        self.maxt = 10000
        self.tdata = [0]
        self.ydata = {0: [2048], 1: [2048], 2: [2048]}
        self.lines = {
            0: Line2D(self.tdata, self.ydata[0], color='C0', label='ADC0'),
            1: Line2D(self.tdata, self.ydata[1], color='C1', label='ADC1'),
            2: Line2D(self.tdata, self.ydata[2], color='C2', label='ADC2'),
        }

        for line in self.lines.values():
            self.ax.add_line(line)

        self.ax.set_autoscaley_on(True)
        self.ax.set_autoscalex_on(True)
        self.ax.legend(loc='upper right')

    def update(self, adc_vals):
        if adc_vals is None:
            return tuple(self.lines.values())

        now = time()
        timestamp = int((now - start_time) * 1000)
        human_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if app.raw_logging:
            app.raw_writer.writerow([human_time, timestamp, adc_vals[0], adc_vals[1], adc_vals[2]])

        if app.custom_logging:
            app.custom_writer.writerow([
                human_time, timestamp, adc_vals[0], adc_vals[1], adc_vals[2],
                app.data_type.get(), app.data_value.get()
            ])

        lastt = self.tdata[-1]
        t = lastt + 1
        self.tdata.append(t)
        for ch in adc_vals:
            self.ydata[ch].append(adc_vals[ch])
            self.lines[ch].set_data(self.tdata, self.ydata[ch])
        self.ax.relim()
        self.ax.autoscale_view()
        if self.tdata:
            xmin = min(self.tdata)
            xmax = max(self.tdata)
            self.ax.set_xlim(xmin, xmax + (xmax - xmin) * 0.05 if xmax > xmin else xmax + 10)


        return tuple(self.lines.values())


def serial_getter():
    buffer = {0: [], 1: [], 2: []}
    last_emit_time = time()
    discard_count = 1

    while True:
        line = app.ser.readline().decode(errors='ignore').strip()
        if line.startswith("ADC"):
            try:
                parts = line.split(":")
                chan = int(parts[0][3])
                val = int(parts[1].strip())
                buffer[chan].append(val)
            except Exception:
                continue

        if time() - last_emit_time >= 1 and all(buffer[ch] for ch in [0, 1, 2]):
            last_emit_time = time()
            if discard_count > 0:
                discard_count -= 1
                buffer = {0: [], 1: [], 2: []}
                continue

            avg_vals = {
                ch: int(sum(buffer[ch]) / len(buffer[ch])) if buffer[ch] else 2048
                for ch in buffer
            }
            yield avg_vals
            buffer = {0: [], 1: [], 2: []}


class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("3-Channel ADC Logger")

        self.raw_logging = False
        self.custom_logging = Fal     
        self.ser = None
        self.custom_file = None
        self.raw_file = None

        # Variables
        self.filename_var = tk.StringVar()
        self.data_type = tk.StringVar()
        self.data_value = tk.StringVar()
        self.com_port = tk.StringVar()

        # COM Port selector
        ttk.Label(self, text="COM Port:").grid(row=0, column=0)
        self.com_ports = [p.device for p in list_ports.comports()]
        self.com_port.set(self.com_ports[0] if self.com_ports else "")
        self.com_dropdown = ttk.Combobox(self, values=self.com_ports, textvariable=self.com_port, state="readonly")
        self.com_dropdown.grid(row=0, column=1, columnspan=2)

        # Logging controls
        ttk.Label(self, text="CSV Filename Prefix:").grid(row=1, column=0)
        ttk.Entry(self, textvariable=self.filename_var).grid(row=1, column=1)
        ttk.Button(self, text="Start Raw Logging", command=self.start_raw).grid(row=1, column=2)
        ttk.Button(self, text="Stop Raw Logging", command=self.stop_raw).grid(row=1, column=3)

        ttk.Label(self, text="Data Type:").grid(row=2, column=0)
        ttk.Entry(self, textvariable=self.data_type).grid(row=2, column=1)
        ttk.Label(self, text="Data Value:").grid(row=2, column=2)
        ttk.Entry(self, textvariable=self.data_value).grid(row=2, column=3)
        ttk.Button(self, text="Start Custom Logging", command=self.start_custom).grid(row=2, column=4)
        ttk.Button(self, text="Stop Custom Logging", command=self.stop_custom).grid(row=2, column=5)

        # Status box
        ttk.Label(self, text="Status:").grid(row=4, column=0, sticky='e')
        self.status_text = tk.StringVar(value="Idle")
        ttk.Label(self, textvariable=self.status_text, foreground="blue").grid(row=4, column=1, columnspan=4, sticky='w')

        # Plot
        fig, ax = plt.subplots()
        self.plotter = Plotter(ax)
        self.canvas = FigureCanvasTkAgg(fig, master=self)
        self.canvas.get_tk_widget().grid(row=3, column=0, columnspan=6)

        self.ani = None
        self.protocol("WM_DELETE_WINDOW", self.on_close)

    def update_status(self, message):
        self.status_text.set(message)

    def init_serial(self):
        try:
            self.ser = serial.Serial(self.com_port.get(), 115200, timeout=1)
            global start_time
            start_time = time()
            if not self.ani:
                self.ani = FuncAnimation(self.plotter.ax.figure, self.plotter.update, serial_getter,
                                         interval=1, blit=True, cache_frame_data=False)
        except Exception as e:
            messagebox.showerror("Serial Error", f"Failed to open serial port: {e}")
            return False
        return True

    def start_raw(self):
        if not self.ser and not self.init_serial():
            return
        filename = self.filename_var.get().strip()
        if not filename:
            self.update_status("Filename required.")
            return

        # Create a unique filename with timestamp to prevent overwrite
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
        raw_filename = f"{timestamp}-{filename}[raw].csv"
        path = os.path.join(SAVE_DIR, raw_filename)

        self.raw_file = open(path, "w", newline='')
        self.raw_writer = csv.writer(self.raw_file)
        self.raw_writer.writerow(["datetime", "ms_elapsed", "ADC0", "ADC1", "ADC2"])
        self.raw_logging = True
        self.update_status(f"Started raw logging to {raw_filename}")

    def stop_raw(self):
        if self.raw_logging and self.raw_file:
            self.raw_file.close()
            self.raw_file = None
        self.raw_logging = False
        self.update_status("Stopped raw logging.")

    def start_custom(self):
        if not self.ser and not self.init_serial():
            return
        filename = self.filename_var.get().strip()
        if not filename:
            self.update_status("Filename required.")
            return
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
        path = os.path.join(SAVE_DIR, f"{timestamp}-{filename}[customfield].csv")
        if self.custom_file is None:
            file_exists = os.path.exists(path)
            self.custom_file = open(path, "a", newline='')
            self.custom_writer = csv.writer(self.custom_file)
            if not file_exists:
                self.custom_writer.writerow(["datetime", "ms_elapsed", "ADC0", "ADC1", "ADC2", "DataType", "DataValue"])
        self.custom_logging = True
        self.update_status("Started custom logging.")

    def stop_custom(self):
        self.custom_logging = False
        self.update_status("Stopped custom logging.")

    def on_close(self):
        self.stop_raw()
        self.stop_custom()
        if self.custom_file:
            self.custom_file.close()
            self.custom_file = None
        if self.ser and self.ser.is_open:
            self.ser.close()
        self.destroy()


# Run the app
#app = App()
#app.mainloop()


## Distance Test

One of the first things I want to know is what hall effect measurements are which distance unit. I took a ruler and marked every 2mm along the opening of the housing for the magnet. I then used a screwdriver to position the magnet shuttle at each stop and recorded the resulting readings. I used the bottom o-ring as the location index of the shuttle since it is easier to see than the edge of the red part. 
```{figure} ../images/jb/2025-04-25-ruler.jpg
:name: 2025-04-25-figure-ruler
Here is the ruler that I marked onto the unit. I used a screwdriver and a zip tie to align the magnet back and forth. I made sure to pull of the screwdriver before measuring that it wouldn't alter the magnetic flux of the magnet.  
```

I went back and forth a couple of times recording the readings at each mark. 

### Distance Test Results

In [4]:
import pandas as pd
import plotly.graph_objects as go

# 1. Load the CSV
df = pd.read_csv('2025-04-25-distance-test-[customfield].csv', parse_dates=['datetime'])

# 2. Bin by DataValue and compute mean ADC values per bin
binned = df.groupby('DataValue').agg({
    'ADC0': 'mean',
    'ADC1': 'mean',
    'ADC2': 'mean'
}).reset_index()

# 3. Plot with Plotly
fig = go.Figure()

# Add a line for each ADC channel
for adc_col in ['ADC0', 'ADC1', 'ADC2']:
    fig.add_trace(go.Scatter(
        x=binned['DataValue'],
        y=binned[adc_col],
        mode='lines+markers',
        name=adc_col
    ))

# 4. Format the plot
fig.update_layout(
    title='ADC Values vs Distance',
    xaxis_title='Distance (mm)',
    yaxis_title='ADC Value',

)

fig.show()
#binned.to_csv('../data/processed/cornell-sensor/2025-04-25-distance-test-binned.csv', index=False)

So based on this, this full range is pretty sensitive. to get maximum sensitivity during pressure test I might want to have the shuttle start at 6mm on my scale. That way I can detect the smallest changes in pressure in the system. 

## Pressure test

pressure test 1 file is just the venting process. Saw a dramatic change in values just from that. not sure why exactly. 
- Actually, this file is empty, I must have forgotten to hit start recording. But basically I just saw the signal of the shuttle moving and not coming back to the original position. The start of the next file shows the starting position anyways, so hopefully it isn't a tragic loss.

pressure test 2 was a basic pressurizing test to failure. I heard a crunch/pop sound around 230 bar (I think, I was wearing my headphones but heard something strange) and then the system stopped sending data. 

```{figure} ../images/jb/2025-04-25-start-position.jpg
:name: 2025-04-25-figure-start-position
The starting position of the shuttle
```
```{figure} ../images/jb/2025-04-25-distance-set.jpg
:name: 2025-04-25-figure-distance-set
marked the back plug so i could get consistent volume on it in future tests
```



```{figure} ../images/jb/2025-04-25-pre-pressure-test.jpg
:name: 2025-04-25-figure-pre-pressure-test
pre pressure test, had it set at 6mm and there was a couple bubbles here and there
```
```{figure} ../images/jb/2025-04-25-post-test.jpg
:name: 2025-04-25-figure-post-test
post test the shuttle had moved a good amount to about 11mm and there weren't any bubbles to be seen
```



```{figure} ../images/jb/2025-04-25-top-post-test.jpg
:name: 2025-04-25-figure-top-post-test
after the test there wasn't anything obviously wrong. There were a couple more holes on the surface, but none of them looked super deep or anything, just as though the surface layer popped and the surface bubble below the surface revealed itself.
```
```{figure} ../images/jb/2025-04-25-post-test-position.jpg
:name: 2025-04-25-figure-post-test-position
Here is an image to keep track of the position for volume calculations later. The end of the shuttle is 28mm from the end of the tube (this for the 11mm position that can be seen above in the photo). Due to refraction at this viewing angle it looks like the bottom of shuttle isn't at 0, but it is if you look at it directly from above.
```







### Results

### Venting air results

In [5]:
import pandas as pd
import plotly.graph_objects as go

# 1. Load the CSV
pt2 = pd.read_csv('2025-04-25-pressure-test-2[raw].csv', parse_dates=['datetime'])

# 2. Bin by DataValue and compute mean ADC values per bin


# 3. Plot with Plotly
fig = go.Figure()

# Add a line for each ADC channel
for adc_col in ['ADC0', 'ADC1', 'ADC2']:
    fig.add_trace(go.Scatter(
        x=pt2['datetime'],
        y=pt2[adc_col],
        mode='lines+markers',
        name=adc_col
    ))

# 4. Format the plot
fig.update_layout(
    title='ADC Values vs Pressure',
    xaxis_title='Pressure (bar)',
    yaxis_title='ADC Value',

)

fig.show()


In [6]:
pressure_slice_ranges = [0,"14:49:30","14:50:43"],[20,"14:51:50","14:52:59"],[40,"14:53:27","14:54:11"],[60,"14:54:44","14:55:30"],[80,"14:55:59","14:57:14"],[100,"14:57:43","14:58:22"],[120,"14:58:48","14:59:29"],[140,"14:59:57","15:00:37"],[160,"15:01:08","15:01:42"]

pt2_slices = []

for pressure, start, end in pressure_slice_ranges:
    # Turn time into datetime with date being 2025-04-25
    start_dt = pd.to_datetime(f"2025-04-25 {start}")
    end_dt = pd.to_datetime(f"2025-04-25 {end}")
    mask = (pt2['datetime'] >= start_dt) & (pt2['datetime'] <= end_dt)
    pt2_slice = pt2.loc[mask].copy()
    pt2_slice["Pressure"] = pressure
    pt2_slices.append(pt2_slice)

# Concatenate all slices into a single DataFrame
pt2_slices = pd.concat(pt2_slices, ignore_index=True)

binned = pt2_slices.groupby('Pressure').agg({
    'ADC0': 'mean',
    'ADC1': 'mean',
    'ADC2': 'mean'
}).reset_index()

# 5. Plot with Plotly
fig = go.Figure()
# Add a line for each ADC channel
for adc_col in ['ADC0', 'ADC1', 'ADC2']:
    fig.add_trace(go.Scatter(
        x=binned['Pressure'],
        y=binned[adc_col],
        mode='lines+markers',
        name=adc_col
    ))

# 4. Format the plot
fig.update_layout(
    title='ADC Values vs Distance',
    xaxis_title='Pressure (bar)',
    yaxis_title='ADC Value',

)
fig.show()
#binned.to_csv("../data/processed/cornell-sensor/2025-04-25-pressure-test-2-binned.csv", index=False)

## Failure Analysis
There isn't any serious destruction externally so I took some cross sections with a band saw. 

```{figure} ../images/jb/2025-04-25-cross-section-1.jpg
:name: 2025-04-25-figure-cross-section-1
on first cross section, no obvious failures anywhere
```
```{figure} ../images/jb/2025-04-25-cross-section-2.jpg
:name: 2025-04-25-figure-cross-section-2
Even on multiple cross sections there wasn't an obvious failure mode. 
```
```{figure} ../images/jb/2025-04-25-cross-section-3.jpg
:name: 2025-04-25-figure-cross-section-3
ones that can be flipped are flipped in this one to see the other side.
```

## Volume Measurements
I filled the vessel up to the top with tap water, and then emptied it in to a beaker and measured the mass of the water to get a volume measurement. 
- Pressure vessel: 247.35g
- 5.15g to fill the end tube 101mm of distance (for volume to distance conversions)
